In [1]:
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn import random_projection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import homogeneity_score
from sklearn.mixture import GaussianMixture
from collections import Counter
import pandas as pd
import numpy as np
import random
from pytictoc import TicToc
t = TicToc()

In [2]:
vocabulary = pd.read_csv("vocab.nytimes.csv",header=None,names=['ID','WORD'] )
vocabulary.shape

(102660, 2)

In [3]:
for ite in range(44442,vocabulary.shape[0]):
    aux_word = vocabulary["WORD"][ite]
    vocabulary["WORD"][ite] = aux_word[4:]

C:\Users\Alvaro\AppData\Local\Temp\ipykernel_18788\417027552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vocabulary["WORD"][ite] = aux_word[4:]


In [4]:
#vocabulary = vocabulary[~vocabulary['WORD'].str.contains('_', na=False, regex=False)]

In [5]:
vocabulary.shape

(102660, 2)

In [6]:
corpus = pd.read_csv("docword.nytimes.txt",header=None,names=['ARTICLE','WORD','OBSERVATIONS'],sep=' ' )

In [7]:
random.seed(19830526)
corpus.shape

(69679427, 3)

In [8]:
basePoliticsWords = ['republican','democratic','senate','congress','senator','representative']
basePoliticsWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(basePoliticsWords)].values.tolist()
articlesPolitics = corpus['ARTICLE'][corpus['WORD'].isin(basePoliticsWordsIDs)].values.tolist()
print(len(articlesPolitics))
selectedPolitics = random.sample(articlesPolitics,2000)
basePoliticsWordsIDs

123418


[7886,
 10185,
 32801,
 32842,
 35020,
 35021,
 55369,
 58011,
 88569,
 88587,
 92424,
 92479]

In [9]:
baseSportWords = ['sport','player','score','competition','championship','mvp']
baseSportWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseSportWords)].values.tolist()
articlesSports = corpus['ARTICLE'][corpus['WORD'].isin(baseSportWordsIDs)].values.tolist()
print(len(articlesSports))
selectedSports = random.sample(articlesSports,2000)
baseSportWordsIDs

115066


[6101, 7518, 25453, 29176, 34560, 37194, 53285, 55276, 81367, 91809]

In [10]:
baseNYWords = ['new_york','manhattan','queens','harlem','brooklyn','new_jersey']
baseNYWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseNYWords)].values.tolist()
articlesNY = corpus['ARTICLE'][corpus['WORD'].isin(baseNYWordsIDs)].values.tolist()
print(len(articlesNY))
selectedNY = random.sample(articlesNY,2000)
baseNYWordsIDs

90905


[4723, 17861, 23359, 51396, 66202, 77181, 82350, 82418]

In [11]:
baseClimateWords = ['climate','hurricane','storm','temperature','snow','rain']
baseClimateWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseClimateWords)].values.tolist()
articlesClimate = corpus['ARTICLE'][corpus['WORD'].isin(baseClimateWordsIDs)].values.tolist()
print(len(articlesClimate))
selectedClimate = random.sample(articlesClimate,2000)
baseClimateWordsIDs

31652


[6826, 19013, 31357, 36511, 37873, 39515, 54738, 67855, 87645, 95253]

In [12]:
baseEducationWords = ['university','graduate','student','school','teacher','learning']
baseEducationWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseEducationWords)].values.tolist()
articlesEducation = corpus['ARTICLE'][corpus['WORD'].isin(baseEducationWordsIDs)].values.tolist()
print(len(articlesEducation))
selectedEducation = random.sample(articlesEducation,2000)
baseEducationWordsIDs

141960


[17033, 22130, 34498, 38097, 39341, 41880, 65170, 95365, 96227, 99200]

In [13]:
baseTechWords = ['software','internet','data','cybersecurity','technology','digital']
baseTechWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseTechWords)].values.tolist()
articlesTech = corpus['ARTICLE'][corpus['WORD'].isin(baseTechWordsIDs)].values.tolist()
print(len(articlesTech))
selectedTech = random.sample(articlesTech,2000)
baseTechWordsIDs

90350


[9422, 9602, 10811, 20570, 36639, 39398, 68584]

In [14]:

selectedArticles = selectedTech + selectedPolitics + selectedNY + selectedClimate + selectedEducation + selectedSports
selectedWords = baseTechWordsIDs + basePoliticsWordsIDs + baseNYWordsIDs + baseClimateWordsIDs + baseEducationWordsIDs + baseSportWordsIDs
print(len(selectedArticles))
#selectedArticles = list(set(selectedArticles))
#print(len(selectedArticles))
eliminating=Counter(selectedArticles)
selectedArticlesSplit = []
for ite in eliminating:
    if eliminating[ite] == 1:
        selectedArticlesSplit.append(ite)
print(len(selectedArticlesSplit))
corpus = corpus[corpus['ARTICLE'].isin(selectedArticles)]
#words_to_keep = vocabulary['ID'].tolist()
#corpus = corpus[corpus['WORD'].isin(words_to_keep)]
corpus.shape
              

12000
11413


(3216090, 3)

In [15]:
corpus = corpus.pivot(index='ARTICLE',columns='WORD',values='OBSERVATIONS')
corpus = corpus.fillna(0)
selectedWords = selectedWords + random.sample(corpus.columns.values.tolist(),10000)    
eliminating=Counter(selectedWords)
selectedWordsSplit = []
for ite in eliminating:
    if eliminating[ite] == 1:
        selectedWordsSplit.append(ite)

In [16]:
print(corpus.shape)
#corpus = corpus.loc[:, (corpus != 0).any(axis=0)]
print(len(selectedWordsSplit))
corpus = corpus[selectedWordsSplit]
print(corpus.shape)

(11702, 80879)
10039
(11702, 10039)


In [17]:

label_real = []
for row in corpus.index:
    if row in selectedPolitics:
        label_real.append(0)
    elif row in selectedTech:
        label_real.append(1)
    elif row in selectedEducation:
        label_real.append(2)
    elif row in selectedNY:
        label_real.append(3)
    elif row in selectedClimate:
        label_real.append(4)
    elif row in selectedSports:
        label_real.append(5)

In [18]:
homogeneity_sol = pd.DataFrame([['Test',0,0,0.000,0.000]],columns=["Algorithm","Components","Clusters","Homogeneity","Time"])
homogeneity_sol

,Algorithm,Components,Clusters,Homogeneity,Time
0,Test,0,0,0.0,0.0


In [19]:
corpus = corpus.fillna(0)
corpus[corpus > 1] = 1
#columnas = list(corpus.columns)
#columnas_words = vocabulary["ID"]=

    

In [20]:
#ICA = FastICA(n_components=200,max_iter=1000)
#corpus_ica=ICA.fit_transform(corpus)

In [21]:
labels_ica = []
for iteCom in range(1,50):
    t.tic()
    ICA = FastICA(n_components=iteCom,max_iter=1000, random_state = 19830526)
    corpus_ica=ICA.fit_transform(corpus)        
    t.toc("ICA calibration for "+str(iteCom)+" components took ")
    for ite in range(1,20):
        t.tic()
        gm_label_ica = GaussianMixture(n_components=ite, random_state=19830526).fit_predict(corpus_ica)         
        labels_ica.append(gm_label_ica)
        taux = t.tocvalue()
        print("ICA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["ICA",iteCom,ite,homogeneity_score(label_real, gm_label_ica),taux]],columns=["Algorithm","Components","Clusters","Homogeneity","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA calibration for 1 components took  862.940463 seconds.
ICA with 1 features and 1 clusters took to run 0.9516682000000856
ICA with 1 features and 2 clusters took to run 0.062471499999901425
ICA with 1 features and 3 clusters took to run 0.05260199999997894
ICA with 1 features and 4 clusters took to run 0.0556642000001375
ICA with 1 features and 5 clusters took to run 0.048051700000087294
ICA with 1 features and 6 clusters took to run 0.056085799999891606
ICA with 1 features and 7 clusters took to run 0.05824170000005324
ICA with 1 features and 8 clusters took to run 0.07788419999997132
ICA with 1 features and 9 clusters took to run 0.07029990000000907
ICA with 1 features and 10 clusters took to run 0.06443379999996068
ICA with 1 features and 11 clusters took to run 0.0770614000000478
ICA with 1 features and 12 clusters took to run 0.08599770000000717
ICA with 1 features and 13 clusters took to run 0.08926160000009986
ICA with 1 features and 14 clusters took to run 0.0877201999999215

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA calibration for 2 components took  839.669758 seconds.
ICA with 2 features and 1 clusters took to run 0.09437900000011723
ICA with 2 features and 2 clusters took to run 0.09083059999989018
ICA with 2 features and 3 clusters took to run 0.11763399999995272
ICA with 2 features and 4 clusters took to run 0.16227889999981926
ICA with 2 features and 5 clusters took to run 0.15480320000006031
ICA with 2 features and 6 clusters took to run 0.1993904000000839
ICA with 2 features and 7 clusters took to run 0.26465700000017023
ICA with 2 features and 8 clusters took to run 0.3280883999998423
ICA with 2 features and 9 clusters took to run 0.34894969999982095
ICA with 2 features and 10 clusters took to run 0.4032042999999703
ICA with 2 features and 11 clusters took to run 0.42577749999986736
ICA with 2 features and 12 clusters took to run 0.4574364999998579
ICA with 2 features and 13 clusters took to run 0.5529225000000224
ICA with 2 features and 14 clusters took to run 0.48403989999997066
ICA

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


...

ICA calibration for 31 components took  830.632961 seconds.
ICA with 31 features and 1 clusters took to run 0.5303560999964247
ICA with 31 features and 2 clusters took to run 4.837571600000956
ICA with 31 features and 3 clusters took to run 0.37959659999614814
ICA with 31 features and 4 clusters took to run 5.330425100000866
ICA with 31 features and 5 clusters took to run 5.8688833999985945
ICA with 31 features and 6 clusters took to run 5.074861699999019
ICA with 31 features and 7 clusters took to run 2.5330570000005537
ICA with 31 features and 8 clusters took to run 11.813513899993268
ICA with 31 features and 9 clusters took to run 12.97180870000011
ICA with 31 features and 10 clusters took to run 13.68651750000572
ICA with 31 features and 11 clusters took to run 4.63473189999786
ICA with 31 features and 12 clusters took to run 4.490694000000076
ICA with 31 features and 13 clusters took to run 6.067070200006128
ICA with 31 features and 14 clusters took to run 5.998574400000507
ICA wi

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA calibration for 32 components took  686.735755 seconds.
ICA with 32 features and 1 clusters took to run 0.16043509999872185
ICA with 32 features and 2 clusters took to run 0.6266422999979113
ICA with 32 features and 3 clusters took to run 1.084664200003317
ICA with 32 features and 4 clusters took to run 0.9628073999992921
ICA with 32 features and 5 clusters took to run 10.865908299994771
ICA with 32 features and 6 clusters took to run 21.51414880000084
ICA with 32 features and 7 clusters took to run 30.543387099998654
ICA with 32 features and 8 clusters took to run 14.216189999999187
ICA with 32 features and 9 clusters took to run 32.13658040000155
ICA with 32 features and 10 clusters took to run 34.915342100001
ICA with 32 features and 11 clusters took to run 30.0967912000051
ICA with 32 features and 12 clusters took to run 32.80043000000296
ICA with 32 features and 13 clusters took to run 27.70838499999809
ICA with 32 features and 14 clusters took to run 20.44369810000353
ICA wit

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


KeyboardInterrupt: 

In [22]:
print(corpus.shape)
print(len(label_real))

(11702, 10039)
11702


In [23]:
homogeneity_sol.to_csv("BagOfWordsClusteringResultsEM_ICA.csv")

In [25]:
df_ica = pd.DataFrame(labels_ica, columns=corpus.index).T
df_ica.to_csv("icaEM2.csv")
